Code for selecting sets of tags and obtaining the PMIDs. Then, these can be tagged in Paperpile for easy access in Boox.

In [1]:
import logging
import numpy as np

import os
os.chdir("..")

import pandas as pd

from src.db import Database
from src.tags import Tags

LOGGER = logging.getLogger(__name__)
logging.basicConfig(level = logging.INFO)

In [2]:
def load_metadata(db, filter=True):
    # Initialize Tag class
    metadata = Tags()

    # Load the tag file and turn into a dataframe
    metadata.load("data/tag_files/tags.yaml")
    md = pd.DataFrame(metadata.tags["tagged_papers"])
    
    # Merge with database
    mddb = db.merge(md, on="id", how="outer")
    
    # List current progress out of total.
    total_papers = mddb.shape[0]
    tagged_papers = sum(~mddb["tag"].isnull())
    percent = np.round((tagged_papers/total_papers) * 100, 2)
    
    LOGGER.info(f"Currently {tagged_papers}/{total_papers} ({percent}%) papers tagged.")
    LOGGER.info(f"{total_papers-tagged_papers} papers remaining.")
        
    mddb = mddb.reset_index()
        
    return mddb

def filter_by_tags(mddb, tag_list: list):
    """ Returns papers containing the tagged term """
    current_tags = mddb["tag"].to_list()
    tags_to_filter = []
    for idx, tags in enumerate(current_tags):
        matches = set(tag_list).intersection(tags)
        if len(matches) == 0:
            pass
        else:
            tags_to_filter.append(idx)
    
    filtered = mddb.iloc[tags_to_filter]
    LOGGER.info(f"{len(filtered)} matches found.")
    return filtered[["id", "tag", "title"]]

In [3]:
# Set tables
tables = ["active_inference", "bayesian_mechanics", "free_energy", "friston", "karl_friston", "predictive_coding", "predictive_processing"]

# Load database
database = Database()
database.load(tables=tables)
db = database.db

# Load metadata
mddb = load_metadata(db, filter=False)

INFO:src.db:Checking tables...
INFO:src.db:Loading tables...
INFO:src.db:Tables downloaded from PubMed on Thursday, Sept. 14, 2023.
INFO:src.tags:YAML tag file successfully loaded from data/tag_files/tags.yaml.
INFO:__main__:Currently 3585/3585 (100.0%) papers tagged.
INFO:__main__:0 papers remaining.


## High-level tag sets

[core ]

**Robotics**

[robotics]
[brain-computer interface, human-machine interface]
[deep active inference]

**Neuropathology and disease**

[addiction]
[Alzheimer's disease, Huntington's disease, down syndrome, multiple sclerosis, Parkinson's disease]
[anxiety, panic disorder]
[apathy, catatonia, depression, mood]
[anosognosia]
[autism spectrum disorder]
[bipolar disorder, borderline personality disorder, delusion, depersonalization, hallucination, obsessive-compulsive disorder, schizophrenia]
[illusion ]
[cancer ]
[clinical psychology ]
[computational psychiatry ]
[disconnection ]
[dyslexia ]
[eating disorder, obesity ]
[epilepsy ]
[headache ]
[hypnosis ]
[hysteria ]
[psychedelics]
[psychoanalysis]
[psychopathology]
[psychophysiology]
[psychotherapy]
[synesthesia]
[Tourette syndrome]

**Behavior**

[animal behavior, anticipation, attachment theory, human behavior]
[attention, precision]
[decision making, goal-directed behavior, motivation, risk]
[exploration, artificial curiosity, information seeking, insight ]
[habit learning, learning, structure learning]
[navigation]

**Perception and general brain function**

[aesthetic perception, anaesthesia, anosognosia, arousal, binocular rivalry]
[audition ]
[vision ]
[cognitive control, economics, rationality]
[belief formation, ]
[default mode network ]
[degeneracy ]
[emotion, empathy ]
[existential threat ]
[intelligence ]
[lateralization ]
[lesions ]
[linguistics ]
[memory ]
[metacognition ]
[mirror-neuron ]
[multimodal integration]
[neural coding]
[neural substrate]
[neuroanatomy]
[neurocircuitry]
[neurocomputation]
[neuroimaging]
[neuromodulation]
[neuronal dynamics]
[neuronal populations]
[nociceptive processing ]
[noise]
[normativity]
[pain]
[perception]
[placebo]
[redundancy]
[resilience]
[salience]
[sleep]
[speech]
[stress]
[stroke]
[timescales]
[time processing]
[uncertainty]
[world model learning]

**Vision**

[object localization]
[saccades]
[scene construction]
[vision]

**Movement and coordination**

[dysmetria ]
[forward model]
[movement, sensorimotor control]
[proprioceptive processing]
[psychogenic tremor]
[reflex arc]

**Consciousness, identity, and selfhood**

[anaesthesia, anosognosia]
[consciousness ]
[integrated information theory ]
[interoceptive processing ]
[meditation ]
[selfhood]

**General biology**

[aging, child development, embryogenesis, morphogenesis]
[immune system, endocrinology, chemical reaction networks, metabolism]
[injury ]

**Agent-environment interaction**

[adaptation, affordances, agent-environment coupling, brain-body interaction, control theory, cybernetics, ecological psychology, embodied cognition]
[niche construction]
[homeostasis / allostasis, autopoiesis, homeostatic reinforcement learning, self-regulation]
[enactive inference, enactivism ]
[energy homeostasis principle ]
[exteroceptive processing ]

**Collective behavior and dynamics**

[ecosystems ] 
[criticality ]
[collective behavior, communication, organizational ]
[culture, social ]
[evolution ]
[mirror-neuron ]
[synchrony]

**New frontiers**

[covid-19 ]
[ethics ]
[military research ]
[quantum mechanics]
[synthetic biology]
[theory of computation]
[translation process research]

**Critique and commentary**

[philosophy]
[editorial]
[comment / response]
[representation]
[theory of mind]

**Theory**

[Bayesian filtering, ]
[Bayesian model reduction, Bayesian model selection]
[Bethe free energy, Kikuchi's free energy, Renyi divergence]
[branching-time active inference ]
[dynamic causal modeling ]
[expected free energy ]
[explainability ]
[free energy of the expected future ]
[generalized free energy ]
[generalized predictive coding ]
[graphical models ]
[hierarchical Gaussian filter ]
[hierarchical model ]
[homeostatic reinforcement learning ]
[information geometry, information theory, expected float entropy ]
[KL control]
[lattice theory]
[logic]
[message passing]
[morphological computation]
[network analysis]
[neuromorphic computing]
[neuronal gauge theory]
[policy pruning]
[projective geometry]
[reinforcement learning]
[signal processing]
[sophisticated inference, sophisticated learning]
[surprisal]
[systems theory]
[temporal model]
[theory]
[time series analysis]
[unsupervised learning]
[variational inference]

**Foundations**

[predictive coding]
[predictive processing]

**Bayesian mechanics**

[Markov blankets, Bayesian mechanics ]
[path-integral formulation]

## Chapter: Theory and Extensions

**Theory**

[Bayesian filtering, ]
[Bayesian model reduction, Bayesian model selection]
[Bethe free energy, Kikuchi's free energy, Renyi divergence]
[branching-time active inference ]
[dynamic causal modeling ]
[expected free energy ]
[explainability ]
[free energy of the expected future ]
[generalized free energy ]
[generalized predictive coding ]
[graphical models ]
[hierarchical Gaussian filter ]
[hierarchical model ]
[homeostatic reinforcement learning ]
[information geometry, information theory, expected float entropy ]
[KL control]
[lattice theory]
[logic]
[message passing]
[morphological computation]
[network analysis]
[neuromorphic computing]
[neuronal gauge theory]
[policy pruning]
[projective geometry]
[reinforcement learning]
[signal processing]
[sophisticated inference, sophisticated learning]
[surprisal]
[systems theory]
[temporal model]
[theory]
[time series analysis]
[unsupervised learning]
[variational inference]

**Equivalence to other fields**

## Chapter: Robotics, planning, and decision-making

**Robotics**

[robotics]
[brain-computer interface, human-machine interface]
[deep active inference]

**Behavior**

[animal behavior, anticipation, attachment theory, human behavior]
[attention, precision]
[decision making, goal-directed behavior, motivation, risk]
[exploration, artificial curiosity, information seeking, insight ]
[habit learning, learning, structure learning]
[navigation]

## Chapter: Neuropathology and disease

(Maybe combine with "General brain and body function" chapter? Title: brain function and dysfunction)

**Neuropathology and disease**

[addiction]
[Alzheimer's disease, Huntington's disease, down syndrome, multiple sclerosis, Parkinson's disease]
[anxiety, panic disorder]
[apathy, catatonia, depression, mood]
[anosognosia]
[autism spectrum disorder]
[bipolar disorder, borderline personality disorder, delusion, depersonalization, hallucination, obsessive-compulsive disorder, schizophrenia]
[illusion ]
[cancer ]
[clinical psychology ]
[computational psychiatry ]
[disconnection ]
[dyslexia ]
[eating disorder, obesity ]
[epilepsy ]
[headache ]
[hypnosis ]
[hysteria ]
[psychedelics]
[psychoanalysis]
[psychopathology]
[psychophysiology]
[psychotherapy]
[synesthesia]
[Tourette syndrome]

## Chapter: Agent-environment interaction and collective behavior

**Agent-environment interaction**

[adaptation, affordances, agent-environment coupling, brain-body interaction, control theory, cybernetics, ecological psychology, embodied cognition]
[niche construction]
[homeostasis / allostasis, autopoiesis, homeostatic reinforcement learning, self-regulation]
[enactive inference, enactivism ]
[energy homeostasis principle ]
[exteroceptive processing ]

**Collective behavior and dynamics**

[ecosystems ] 
[criticality ]
[collective behavior, communication, organizational ]
[culture, social ]
[evolution ]
[mirror-neuron ]
[synchrony]

## Chapter: General brain and body function

**Perception and general brain function**

[aesthetic perception, anaesthesia, anosognosia, arousal, binocular rivalry]
[audition ]
[vision ]
[cognitive control, economics, rationality]
[belief formation, ]
[default mode network ]
[degeneracy ]
[emotion, empathy ]
[existential threat ]
[intelligence ]
[lateralization ]
[lesions ]
[linguistics ]
[memory ]
[metacognition ]
[mirror-neuron ]
[multimodal integration]
[neural coding]
[neural substrate]
[neuroanatomy]
[neurocircuitry]
[neurocomputation]
[neuroimaging]
[neuromodulation]
[neuronal dynamics]
[neuronal populations]
[nociceptive processing ]
[noise]
[normativity]
[pain]
[perception]
[placebo]
[redundancy]
[resilience]
[salience]
[sleep]
[speech]
[stress]
[stroke]
[timescales]
[time processing]
[uncertainty]
[world model learning]

**Vision**

[object localization]
[saccades]
[scene construction]
[vision]

**Movement and coordination**

[dysmetria ]
[forward model]
[movement, sensorimotor control]
[proprioceptive processing]
[psychogenic tremor]
[reflex arc]

**General biology**

[aging, child development, embryogenesis, morphogenesis]
[immune system, endocrinology, chemical reaction networks, metabolism]
[injury ]

## Chapter: The future of active inference

**Consciousness, identity, and selfhood**

[anaesthesia, anosognosia]
[consciousness ]
[integrated information theory ]
[interoceptive processing ]
[meditation ]
[selfhood]

**New frontiers**

[covid-19 ]
[ethics ]
[military research ]
[quantum mechanics]
[synthetic biology]
[theory of computation]
[translation process research]

**Critique and commentary**

[philosophy]
[editorial]
[comment / response]
[representation]
[theory of mind]

In [4]:
# Tag options for Robotics, Planning, and Decision-Making
# tag_list = [
#     ["animal behavior", "anticipation", "attachment theory", "human behavior"],
#     ["attention", "precision"],
#     ["decision making", "goal-directed behavior", "motivation", "risk"],
#     ["exploration", "artificial curiosity", "information seeking", "insight"],
#     ["habit learning", "learning", "structure learning"],
#     ["navigation"]]

# tag_list = [["robotics"]]
# tag_list = [["brain-computer interface", "human-machine interface"]]
# tag_list = [["deep active inference"]]

# tag_list = [
#     ["Bayesian filtering"],
#     ["Bayesian model reduction", "Bayesian model selection"],
#     ["Bethe free energy", "Kikuchi's free energy", "Renyi divergence"],
#     ["branching-time active inference"],
#     ["dynamic causal modeling"],
#     ["expected free energy"],
#     ["explainability"],
#     ["free energy of the expected future"],
#     ["generalized free energy"],
#     ["generalized predictive coding"],
#     ["graphical models"],
#     ["hierarchical Gaussian filter"],
#     ["hierarchical model"],
#     ["homeostatic reinforcement learning"],
#     ["information geometry", "information theory"," expected float entropy"],
#     ["KL control"],
#     ["lattice theory"],
#     ["logic"],
#     ["message passing"],
#     ["morphological computation"],
#     ["network analysis"],
#     ["neuromorphic computing"],
#     ["neuronal gauge theory"],
#     ["policy pruning"],
#     ["projective geometry"],
#     ["reinforcement learning"],
#     ["signal processing"],
#     ["sophisticated inference", "sophisticated learning"],
#     ["surprisal"],
#     ["systems theory"],
#     ["temporal model"],
#     ["theory"],
#     ["time series analysis"],
#     ["unsupervised learning"],
#     ["variational inference"]
# ]

tag_list = [
    ["anaesthesia"],
    ["anosognosia"],
    ["consciousness"],
    ["integrated information theory"],
    ["interoceptive processing"],
    ["meditation"],
    ["selfhood"],
    ["covid-19"],
    ["ethics"],
    ["military research"],
    ["quantum mechanics"],
    ["synthetic biology"],
    ["theory of computation"],
    ["translation process research"],
    ["philosophy"],
    ["editorial"],
    ["comment / response"],
    ["representation"],
    ["theory of mind"],
]


tag_list = [elem for sublist in tag_list for elem in sublist]

filtered = filter_by_tags(mddb, tag_list=tag_list)
filtered = filtered["title"].to_list()
filtered = list(set(filtered))
LOGGER.info(f"{len(filtered)} items after dropping duplicates.")
# filtered

INFO:__main__:435 matches found.
INFO:__main__:434 items after dropping duplicates.


In [5]:
filtered

['Depersonalization disorder as a systematic downregulation of interoceptive signals',
 'Can you tickle yourself if you swap bodies with someone else?',
 'Neural Mechanisms and Psychology of Psychedelic Ego Dissolution',
 'Commentary: Respiration-Entrained Brain Rhythms Are Global but Often Overlooked',
 'Against free energy, for direct perception',
 'Passive motion and active inference: Commentary on "Muscleless motor synergies and actions without movements: From motor neuroscience to cognitive robotics" by Vishwanathan Mohan, Ajaz Bhat and Pietro Morasso',
 'The theory of constructed emotion: an active inference account of interoception and categorization',
 'Regarding flows under the free energy principle: A comment on "How particular is the physics of the free energy principle?" by Aguilera, Millidge, Tschantz, and Buckley',
 'Does the free energy principle sleep on it?: Comment on "How particular is the physics of the free energy principle?" by Miguel Aguilera et al',
 'Fetal brai

To do:
- Change the filtering code to an actual function
- Create a function which returns overlaps between tags. This will be useful as a way of determining whether or not there have been papers already described elsewhere in another to avoid redoing work.